# An effort to write GSLR out of the OmicsIntegrator Framework

In [1]:
%pylab inline
import sys
import pickle
import pandas as pd

from sklearn.preprocessing import LabelEncoder

from matplotlib_venn import venn3, venn3_circles, venn2

repo_path = '/Users/alex/Documents/gslr/'
data_path = repo_path + 'experiments/generated_data/3/'
KEGG_path = repo_path + 'experiments/KEGG/KEGG_df.filtered.with_correlates.pickle'
interactome_path = repo_path + 'experiments/algorithms/pcsf/inbiomap_temp.tsv'

pathway_id = 'hsa04110'

sys.path.append('/Users/alex/Documents/OmicsIntegrator2/src/')
from graph import Graph

sys.path.append(repo_path + 'gslr/')
import gslr

Populating the interactive namespace from numpy and matplotlib


### I. Load Synthetic Dataset which is known to be at least somewhat connected in the interactome. 

In [2]:
dataset = pd.read_csv(data_path + pathway_id + '_inbiomap_exp.csv', index_col=0)
dataset.head()

,ZNF91,NDEL1,ELAVL1,SUMO1,SUMO3,CHMP5,UBC,HTT,E2F4,ACP5,...,SPANXN4,ZNF605,SERPINB10,ANKAR,RRH,DHH,CYSLTR1,ZNF268,COL23A1,MEDAG
hsa04110,0.0,0.115272,-0.365345,-0.014955,-0.374350,0.109953,-0.0,-0.313725,0.034973,-0.309654,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
hsa04110,-0.0,-0.138520,-0.340004,0.430427,0.618810,-0.400398,-0.0,0.281479,-0.903482,0.312078,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
negative,-0.0,-0.492418,0.612346,0.544840,-0.253648,-0.004268,-0.0,-0.109864,0.337787,-0.604446,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
hsa04110,-0.0,-0.592521,0.050179,0.383061,0.261450,-0.131206,0.0,-0.265055,0.179607,-0.416877,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
hsa04110,0.0,0.000973,0.040952,-0.728286,-0.604990,-0.119933,-0.0,-0.047649,0.165359,-0.616325,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### II. Extract Labels 

In [3]:
labeler = LabelEncoder()
labeler.fit(dataset.index.tolist())
labels = labeler.transform(dataset.index.tolist())
y = labels
y

array([0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0])

### III. Graph Sparse Logistic Regression

In [3]:
graph = Graph(interactome_path, {'a':1500, 'b':4, 'w':7})

06:45:35 - Graph: INFO - Duplicated edges in the interactome file (we'll keep the max cost):
06:45:35 - Graph: INFO - []


In [ ]:
def gslr(graph, X, y, learning_rate, iterations):

    theta = np.zeros(len(graph.nodes))

    for i in range(iterations):

        gradients = gradient_descent(X.transpose(), theta, y)

        theta = theta - (learning_rate * gradients)

        vertex_indices, edge_indices = graph.pcsf(theta)

        theta[~vertex_indices] = 0

    return theta


def gradient_descent(X, theta, y):

    denominators = 1.0 + np.exp(np.multiply(y, np.dot(X, theta)))
    return -np.sum(np.divide(np.multiply(np.expand_dims(y, axis=1), X), np.expand_dims(denominators, axis=1)), axis=0)

